In [44]:
#!pip install spacy
    
import spacy
import pickle
import random

In [70]:
train_data = pickle.load(open('train_data.pkl','rb')) 

In [46]:
train_data[0]

('Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?isid=rex-

In [79]:
# Load the model
nlp= spacy.blank('en')

# Function to train model
def train_model(train_data):
    if 'ner' not in nlp.pipe_names: #named entity recognition
        ner=nlp.create_pipe('ner') # add ner pipe
        nlp.add_pipe("ner", last= True)
    
    #loop over train_data
    for _,annotation in train_data:
        for ent in annotation['entities']:
            ner.add_label(ent[2]) #add labels
    
    # disable other pipelines
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Statring iteration " + str(itn))
            random.shuffle(train_data)
            losses = {}
            index = 0
            for text, annotations in train_data:
                try:
                    nlp.update(
                        [text],  # batch of texts
                        [annotations],  # batch of annotations
                        drop=0.2,  # dropout - make it harder to memorise data
                        sgd=optimizer,  # callable to update weights
                        losses=losses)
                except Exception as e:
                    pass
         
    print(losses)  
    

In [80]:
train_model(train_data)

Statring iteration 0
Statring iteration 1
Statring iteration 2
Statring iteration 3
Statring iteration 4
Statring iteration 5
Statring iteration 6
Statring iteration 7
Statring iteration 8
Statring iteration 9
{}


In [81]:
# Analyze model
nlp.to_disk('nlp_model_2')


In [82]:
nlp_model = spacy.load('nlp_model_2')


In [51]:
train_data[0][0]


'Vishwanath P Senior Executive (MIS & Audit) - Job Profile in LabourNet Services India Pvt Ltd  Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Vishwanath-P/06a16ac2d087d3c9  I look forward to a career that provides me an opportunity to improve my knowledge and personality in a challenging work atmosphere. In the long run, I would like to grow leadership position in the organization and make a meaningful contribution to the organization.  Overview: ➢ Total experience of 7.3years. ➢ Have maintained superior\'s and peer\'s confidence in efficiency and accuracy of my work.  WORK EXPERIENCE  Senior Executive (MIS & Audit)  Job Profile in LabourNet Services India Pvt Ltd -  June 2016 to Present  from June 2016.  Operating System: Windows 7 Tools Used: MS Office (Excel, Word, PowerPoint, Outlook) ➢ Provide weekly targets to Placement officers and follow-up on the targets v/s achieved and more importantly all the placement related activities are documented. ➢ Working on projection aga

In [86]:
#Print classification of data
doc = nlp_model(train_data[0][0])
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')
